In [2]:
using Pkg
Pkg.activate(".")

  Activating new environment at `~/Documents/Github2/Physics-215-Julia/Session 1 - HPC and the Julia Framework/Project.toml`


### KR1: Use ```@code_*``` to examine a simple function. The `*` is replaceable by `native`, `typed`, `warntype`, and others. Discover them.

Let us first define a simple function. Written below is a function that takes two variables and outputs their sum.

In [3]:
function f(x,y) x + y end

f (generic function with 1 method)

In [4]:
@code_native f(1,2)

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ In[3]:1 within `f'
; │┌ @ int.jl:87 within `+'
	leaq	(%rdi,%rsi), %rax
; │└
	retq
	nopw	%cs:(%rax,%rax)
; └


In [5]:
@code_typed f(1.1,2)

CodeInfo(
1 ─ %1 = Base.sitofp(Float64, y)::Float64
│   %2 = Base.add_float(x, %1)::Float64
└──      return %2
) => Float64

In [6]:
@code_warntype f(1.1,2)

Variables
  #self#::Core.Const(f)
  x::Float64
  y::Int64

Body::Float64
1 ─ %1 = (x + y)::Float64
└──      return %1


In [7]:
@code_lowered f(1,2)

CodeInfo(
1 ─ %1 = x + y
└──      return %1
)

In [8]:
@code_llvm f(1,2)

;  @ In[3]:1 within `f'
define i64 @julia_f_1908(i64 signext %0, i64 signext %1) {
top:
; ┌ @ int.jl:87 within `+'
   %2 = add i64 %1, %0
; └
  ret i64 %2
}


Now all of these `macro` functions are new to me so I will just to explain what I understood from our class discussion and some basic googling. These `macro` functions simply convert your `code` into another form of `code` that has a different expression that I believe is more readable by the machine. Using these `@code_*` macros also show us for example, the data types of our inputs, the step-by-step process inside the function and what the output is. In Julia, there are 5 `@code_*` marcos which are `native`, `typed`, `warntype`, `lowered`, and `llvm`. These difference macro functions are I believe just different formms of how the code is being implemented by Julia.

### KR2: Demonstrate that Julia is able to determine constants in codes.

Let us create a new function `g(x)` that takes a single number then performs a sequence of multiplication and division

In [42]:
function g(x)
    a = sqrt(5) * x
    return exp(2) / a
end

g (generic function with 1 method)

In [43]:
@code_llvm  g(1.1)

;  @ In[42]:1 within `g'
define double @julia_g_2003(double %0) {
top:
;  @ In[42]:2 within `g'
; ┌ @ float.jl:332 within `*'
   %1 = fmul double %0, 0x4001E3779B97F4A8
; └
;  @ In[42]:3 within `g'
; ┌ @ float.jl:335 within `/'
   %2 = fdiv double 0x401D8E64B8D4DDAE, %1
; └
  ret double %2
}


Now the important part of the result of our `@code_llvm` are the following:
```
%1 = fmul double %0, 0x4001E3779B97F4A8
%2 = fdiv double 0x401D8E64B8D4DDAE, %1
```
Now we see that `%1` represents the first line of our function which is `sqrt(5) * x`. We also see that the constant `sqrt(5)` has been stored in a memory location with value `0x4001E3779B97F4A8`. Now what Julia does is if it sees an expression that does not have a variable where it can be simplified into a single constant value, that value will be stored somewhere so that every time that specific value is needed, Julia will just pull the value from the storage instead of computing it again. This can also be seen in the next line where the value of `exp(2)` is now represented by `0x401D8E64B8D4DDAE`.

### KR3: Demonstrate Julia's type-inference and multiple dispatch.